# 实验 4.1：训练数据提取攻击

## 🎯 学习目标

完成本实验后，你将能够：
- ✅ 解释语言模型的"记忆"现象与过拟合的关系
- ✅ 理解困惑度（Perplexity）作为隐私泄露指标的含义
- ✅ 实现基于补全的训练数据提取攻击
- ✅ 分析不同提示策略对提取效果的影响

## 📚 前置知识

- 大语言模型的基本工作原理（输入、输出、概率分布）
- 交叉熵损失和困惑度的概念
- Python 数据处理和可视化基础

## 🖥️ 实验环境

| 项目 | 说明 |
|------|------|
| 平台 | 腾讯 Cloud Studio |
| GPU | NVIDIA Tesla T4（16GB）|
| 模型 | uer/gpt2-chinese-cluecorpussmall（中文 GPT-2）|
| 依赖 | torch>=2.0, transformers>=4.30 |

## ⏱️ 预计时间：30 分钟

## 📝 填空说明

本实验共 **4 个填空**，难度：⭐⭐⭐☆☆

⚠️ **安全提醒**：本实验仅用于教育目的，演示如何防护模型隐私泄露。请勿将技术用于未授权系统。

## 第一部分：环境准备与概念回顾

### 核心概念：什么是"训练数据提取"？

**类比理解**：
- 你背诵一首诗，当别人给你"床前明月光"时，你可能会自动接"疑是地上霜"
- 语言模型也是如此——训练数据中出现过的内容可能被"记住"
- 攻击者可以通过精心设计的提示"诱导"模型输出这些记忆内容

**隐私风险**：
- 训练数据可能包含个人信息（邮箱、电话、地址、密码）
- 模型可能无意中泄露这些敏感信息
- 研究表明大型语言模型可以逐字记住训练集的部分内容

### 关键指标：困惑度（Perplexity）

困惑度衡量模型对文本的"熟悉"程度：
- **低困惑度**：模型很熟悉（可能见过/记住了）
- **高困惑度**：模型不熟悉（需要泛化能力）

计算公式：`Perplexity = exp(交叉熵损失)`

In [ ]:
### 第一步：安装依赖

# 本实验所需依赖
!pip install torch>=2.0 transformers>=4.30 -q

# 验证安装
import torch
import transformers
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print("✓ 依赖安装成功！")

# 导入其他必要库
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("\n设置完成，准备加载模型...")

In [ ]:
### 第二步：加载模型

print("正在加载中文 GPT-2 模型（约 350MB）...")
print("⏳ 首次加载可能需要 1-2 分钟...\n")

# 使用中文 GPT-2 模型
model_name = "uer/gpt2-chinese-cluecorpussmall"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.eval()
    
    # 设置 pad_token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print("=" * 50)
    print("✓ 模型加载完成！")
    print("=" * 50)
    print(f"模型大小：{model_name}")
    print(f"词表大小：{len(tokenizer)} tokens")
    print(f"模型参数量：约 1.3B")
    
except Exception as e:
    print(f"❌ 模型加载失败：{e}")
    print("请检查网络连接或稍后重试")

### ✅ 检查点

运行到这里，你应该看到：
- [x] 依赖安装成功，显示 PyTorch 和 Transformers 版本
- [x] 模型加载完成，无错误提示
- [x] 显示模型基本信息（参数量约 1.3B）

如果遇到问题，请检查：
- 是否选择了 GPU 环境
- 网络是否能访问 Hugging Face

In [ ]:
### 第三步：定义文本生成函数

def generate_text(prompt, max_length=50, num_return=1, temperature=1.0):
    """
    根据提示生成文本
    
    参数:
        prompt: 输入提示文本
        max_length: 最大生成长度（包括输入）
        num_return: 返回几个生成结果
        temperature: 温度参数（越低越确定性，推荐 0.7-1.5）
    
    返回:
        生成文本列表
    """
    # 编码输入
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    
    # 生成文本
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            num_return_sequences=num_return,
            temperature=temperature,
            do_sample=True,             # 启用采样
            top_p=0.9,                   # nucleus采样
            pad_token_id=tokenizer.pad_token_id
        )
    
    # 解码输出
    results = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        results.append(text)
    
    return results


# 测试生成函数
print("测试文本生成功能...")
test_prompt = "你好，我的名字是"
test_result = generate_text(test_prompt, max_length=30)
print(f"\n输入提示：{test_prompt}")
print(f"模型生成：{test_result[0]}")
print("\n✓ 文本生成功能正常！")

## 第二部分：计算困惑度（Perplexity）

### 什么是困惑度？

困惑度是衡量模型对文本"熟悉程度"的指标：

| 困惑度范围 | 含义 | 可能原因 |
|-----------|------|---------|
| < 10 | 非常熟悉 | 训练数据中的高频内容 |
| 10-50 | 较熟悉 | 常见模式或类似内容 |
| 50-200 | 一般 | 正常泛化能力 |
| > 200 | 不熟悉 | 稀有内容或随机文本 |

**直觉理解**：如果模型对某段文本困惑度很低，说明它在训练时可能"见过"这段文本！

In [ ]:
# ========== 填空 1：实现困惑度计算 ==========
# 
# 🎯 任务：完成困惑度计算函数
# 
# 💡 提示：
#   - 困惑度 = exp(平均交叉熵损失)
#   - 使用 np.exp() 计算指数
#   - 变量 loss 已经是平均损失
# 
# 📖 参考：
#   Perplexity 越低 → 模型对文本越熟悉
# 
# 请将 ___________ 替换为正确的代码

def calculate_perplexity(text):
    """
    计算文本的困惑度
    
    参数:
        text: 要计算的文本字符串
    
    返回:
        困惑度值（float）
    """
    # 编码文本
    inputs = tokenizer.encode(text, return_tensors='pt')
    
    # 计算损失（labels=inputs 表示语言建模任务）
    with torch.no_grad():
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss.item()  # 平均交叉熵损失
    
    # ========== 填空位置 ==========
    # 计算困惑度：perplexity = exp(loss)
    perplexity = ___________  # 期望：使用 np.exp(loss)
    
    return perplexity


# 测试困惑度计算
test_texts = [
    "中华人民共和国是一个伟大的国家。",  # 常见句子（可能低困惑度）
    "啊斯蒂芬库里咖喱味道真好吃呀。",    # 随机内容（可能高困惑度）
]

print("困惑度测试：")
print("=" * 50)
for text in test_texts:
    ppl = calculate_perplexity(text)
    print(f"文本：{text}")
    print(f"困惑度：{ppl:.2f}\n")

In [ ]:
### 🤔 思考一下

1. **观察**：哪个句子的困惑度更低？为什么？
2. **分析**：困惑度低意味着什么？这与模型的训练有什么关系？

## 第三部分：基于补全的提取攻击

### 攻击原理

1. **设计诱导性提示**：使用可能在训练数据中出现的格式（如联系方式、地址等）
2. **生成多个补全**：让模型产生多个可能的续文
3. **计算困惑度**：困惑度低的补全更可能来自训练数据
4. **评估隐私泄露**：汇总得到可疑的敏感信息

### 演示：设计提示

# ========== 填空 2：设计提取提示 ==========
# 
# 🎯 任务：选择一个可能触发模型记忆的提示
# 
# 💡 提示：
#   - 模型可能会"记住"训练数据中常见的模板
#   - 格式化内容（如"邮箱地址"、"电话号码"）更容易被记住
#   - 著名文学作品（如古诗词）通常有低困惑度
# 
# 难度：⭐⭐☆☆☆
# 
# 请选择下列选项中的一个替换 ___________

# 示例：以下提示可能触发模型记忆
extraction_prompts = [
    ___________,           # 填空 2：选择一个文学作品的开头
    "我的电话号码是",
    "邮箱地址是",
    "我最喜欢的古诗是",
]

print("设计的提取提示：")
print("=" * 50)
for i, prompt in enumerate(extraction_prompts):
    if prompt:  # 跳过空的填空
        print(f"{i+1}. '{prompt}'")
    else:
        print(f"{i+1}. [待填空]")

In [ ]:
### 执行提取攻击

print("尝试提取可能的训练数据...")
print("=" * 50)

for prompt in extraction_prompts:
    if prompt:  # 跳过空的填空
        print(f"\n提示: '{prompt}'")
        print("-" * 50)
        
        # 生成 3 个补全结果
        completions = generate_text(prompt, max_length=40, num_return=3, temperature=0.7)
        
        for i, comp in enumerate(completions, 1):
            # 只显示生成的部分
            generated = comp[len(prompt):].strip()
            
            # 计算困惑度
            ppl = calculate_perplexity(comp)
            
            print(f"  补全 {i} [困惑度={ppl:5.1f}]: {generated[:40]}")

In [ ]:
## 第四部分：记忆 vs 泛化分析

### 实验设计

我们将比较两类文本的困惑度：
1. **可能被"记住"的文本**：著名文学作品、常识性事实
2. **需要"泛化"的文本**：新创作的、不太可能出现在训练集中的内容

# ========== 填空 3：对比两类文本 ==========
# 
# 🎯 任务：计算"记忆"文本的困惑度列表
# 
# 💡 提示：
#   - 使用列表推导式 [function(item) for item in list]
#   - 对 memorized_texts 中的每个文本调用 calculate_perplexity
#   - 参考：[calculate_perplexity(t) for t in memorized_texts]
# 
# 难度：⭐⭐☆☆☆

# 可能被"记住"的著名文本
memorized_texts = [
    "床前明月光，疑是地上霜。",  # 李白 - 静夜思
    "中华人民共和国成立于一九四九年。",  # 历史常识
    "北京是中国的首都。",  # 地理常识
]

# 新创作的、不太可能在训练集中的文本
novel_texts = [
    "紫色的大象在彩虹桥上跳舞。",
    "我最喜欢的早餐是量子物理配吐司。",
    "那个机器人决定成为一名专业的云彩画家。",
]

print("记忆 vs 泛化 - 困惑度对比")
print("=" * 50)

# ========== 填空位置 ==========
# 计算"记忆"文本的困惑度
mem_ppls = ___________  # 期望：[calculate_perplexity(t) for t in memorized_texts]

# 计算"泛化"文本的困惑度
nov_ppls = [calculate_perplexity(t) for t in novel_texts]

print("\n著名文本（可能被记忆）：")
for text, ppl in zip(memorized_texts, mem_ppls):
    print(f"  困惑度 {ppl:6.2f}: {text[:25]}...")

print("\n新创文本（需要泛化）：")
for text, ppl in zip(novel_texts, nov_ppls):
    print(f"  困惑度 {ppl:6.2f}: {text[:25]}...")

print(f"\n平均困惑度对比：")
print(f"  著名文本: {np.mean(mem_ppls):.2f}")
print(f"  新创文本: {np.mean(nov_ppls):.2f}")
print(f"  差异: {np.mean(nov_ppls) - np.mean(mem_ppls):.2f}")

# 可视化对比
plt.figure(figsize=(10, 5))

x = np.arange(3)
width = 0.35

plt.bar(x - width/2, mem_ppls, width, label='著名文本（可能被记忆）', color='#FF6B6B', alpha=0.8)
plt.bar(x + width/2, nov_ppls, width, label='新创文本（需要泛化）', color='#4ECDC4', alpha=0.8)

plt.xlabel('样本号', fontsize=11, fontweight='bold')
plt.ylabel('困惑度', fontsize=11, fontweight='bold')
plt.title('记忆 vs 泛化：困惑度对比\n（低困惑度 = 模型更"熟悉"）', fontsize=13, fontweight='bold')
plt.xticks(x, ['样本1', '样本2', '样本3'])
plt.legend(fontsize=10)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n🔍 关键观察：")
print(f"  著名文本困惑度通常较低（< 20），说明模型很"熟悉"")
print(f"  新创文本困惑度通常较高（> 50），说明模型需要"泛化"")
print(f"\n💡 含义：")
print(f"  这种困惑度差异可能被用于识别训练数据中的内容！")

In [ ]:
## 第五部分：温度参数对提取的影响

### 温度参数的作用

温度（Temperature）控制生成的随机性：
- **低温度（0.3-0.7）**：更确定性，倾向于输出高概率内容（可能是记忆）
- **高温度（1.0-1.5）**：更随机，输出更多样化

**隐私含义**：低温度可能更容易提取训练数据！

# ========== 填空 4：测试不同温度 ==========
# 
# 🎯 任务：观察不同温度对生成内容的影响
# 
# 💡 提示：
#   - temperature 参数在 generate_text 函数中
#   - 完成填空 2 后，extraction_prompts[0] 就是你选的提示
# 
# 难度：⭐☆☆☆☆

# 选择一个提示进行测试
test_prompt = "床前明月光，"  # 或使用 extraction_prompts[0]

# 测试不同温度
temperatures = [0.5, 1.0, 1.5]

print(f"提示: '{test_prompt}'")
print("=" * 50)
print("不同温度的生成效果对比")
print("=" * 50)

for temp in temperatures:
    print(f"\n🌡️ 温度 = {temp}:")
    print("-" * 50)
    
    # 生成 3 个补全
    completions = generate_text(test_prompt, max_length=40, num_return=3, temperature=temp)
    
    for i, comp in enumerate(completions, 1):
        generated = comp[len(test_prompt):].strip()
        ppl = calculate_perplexity(comp)
        print(f"  {i}. [困惑度={ppl:5.1f}] {generated[:35]}")

In [ ]:
print("\n💡 观察：")
print("  低温度 → 生成更一致 → 更多低困惑度内容 → 更容易提取训练数据")
print("  高温度 → 生成更多样 → 困惑度波动大 → 提取难度增加")

## 📋 实验小结

### 核心收获

1. **概念理解**：
   - 语言模型可能会"记住"训练数据中的内容
   - 困惑度低 = 模型熟悉 = 可能是训练数据

2. **技能掌握**：
   - 能计算困惑度评估文本"熟悉程度"
   - 能设计提示诱导模型生成特定内容
   - 能识别哪些内容最容易被提取

3. **安全认识**：
   - 模型隐私泄露是真实的安全威胁
   - 攻击者可通过合理设计的提示提取敏感信息
   - 防御需要从数据清洗、训练方法和输出过滤等多方面入手

### 关键代码回顾

```python
# 困惑度计算（隐私泄露指标）
perplexity = np.exp(loss)

# 列表推导式（批量计算）
ppls = [calculate_perplexity(t) for t in texts]

# 温度参数影响生成确定性（低温→更容易泄露记忆）
generate_text(prompt, temperature=0.7)
```

### 🤔 思考与讨论

1. **观察**：在实验中，哪类提示最容易触发模型的记忆？为什么？

2. **分析**：困惑度与过拟合有什么关系？一个过拟合的模型是否更容易被提取攻击？

3. **应用**：如果你是模型部署者，如何防止类似攻击？
   - 数据预处理：移除敏感信息
   - 训练方法：使用差分隐私等技术
   - 输出过滤：检测并过滤敏感信息格式

4. **扩展**：除了困惑度，还有其他指标可以用于检测隐私泄露吗？

### 延伸阅读

- [论文] Carlini et al. "Extracting Training Data from Large Language Models" (USENIX Security 2021)
- [进阶实验] 实验 4.2：成员推理攻击 - 判断样本是否在训练集中
- [防御方法] 实验 4.3：差分隐私训练 - 使用噪声保护隐私

---

## 参考答案

<details>
<summary>点击查看参考答案</summary>

**填空 1**：困惑度计算
```python
perplexity = np.exp(loss)
```

**填空 2**：选择提取提示（示例答案，可多种选择）
```python
"床前明月光，"  # 李白的诗句
# 或
"从前有座山，"  # 民间故事开头
```

**填空 3**：计算困惑度列表
```python
mem_ppls = [calculate_perplexity(t) for t in memorized_texts]
```

**填空 4**：温度参数理解（无需填空，观察输出即可）
- 低温度生成更一致，困惑度通常更低
- 高温度生成更多样，困惑度波动更大

</details>